In [1]:
import openai
import os

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
openai.api_key = OPENAI_API_KEY

def ask_gpt(model, system_prompt, question, conversation_history=None, max_history=3):
    if conversation_history is None:
        conversation_history = []

    # Truncate conversation_history if it exceeds the limit
    if len(conversation_history) > max_history:
        conversation_history = conversation_history[-max_history:]

    conversation = [
        {"role": "system", "content": system_prompt},
        *conversation_history,
        {"role": "user", "content": question}
    ]

    response = openai.ChatCompletion.create(
        model=model,
        messages=conversation,
        max_tokens=150,                 # one token is roughly 4 characters
        temperature=1.0,
        top_p=1,
    )

    assistant_response = response['choices'][0]['message']['content']
    
    # Update the conversation history
    conversation_history.append({"role": "assistant", "content": assistant_response})
    conversation_history.append({"role": "user", "content": question})

    return assistant_response, conversation_history


In [ ]:

# Usage example
model_name = "gpt-3.5-turbo"
system_prompt = "You are a helpful assistant."
question = "What is the capital of New Zealand?"

response, convo_history = ask_gpt(model_name, system_prompt, question)
print(response)

question = "What were my last two questions?"
response, convo_history = ask_gpt(model_name, system_prompt, question, conversation_history=convo_history)
print(response)

In [ ]:
import os
import openai
import discord
import nest_asyncio
from discord.ext import commands

# Apply the nest_asyncio patch
nest_asyncio.apply()

DISCORD_TOKEN = os.getenv('DISCORD_TOKEN')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
TARGET_CHANNEL_ID = int(os.getenv('DISCORD_CHANNEL_ID'))
openai.api_key = OPENAI_API_KEY

intents = discord.Intents.default()
intents.messages = True
intents.message_content = True


class ChatBot(discord.Client):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.model = "gpt-3.5-turbo"
        self.character = "You are a helpful assistant. Include emojis."
        self.conversation_history = []  # Add this line to define the conversation_history attribute
       

    async def on_ready(self):
        print(f'We have logged in as {self.user}')
        target_channel_id = TARGET_CHANNEL_ID 
        target_channel = self.get_channel(target_channel_id)
        if target_channel:
            await target_channel.send("`Lukebot` is online  💅 Type `/help` for a list of commands.")
        else:
            print("Could not find the target channel.")
        

    async def on_message(self, message):
        if message.author == self.user:
            return

    async def on_message(self, message):
        if message.author == self.user:
            return

        print(f"incoming: {message.content}")

        if message.content.startswith("/help") or message.content.startswith("/info"):
            # respond with current config
            await message.channel.send(
                f"**Lukebot Config**\nModel: {self.model}\n"
                f"```fix\n"
                f"Character: {self.character}\n"
                f"```\n"
                f"Type `/character` and a description to change character.")

        elif message.content.startswith("/character "):
            self.character = message.content[len("/character "):]
            await message.channel.send(f"✅ Character updated: \n"
                                       f"```fix\n"
                                       f"Character: {self.character}\n"
                                       f"```")
            
        elif message.content.startswith("/"):
            await message.channel.send("Type `/help` for a list of commands.")
            
        else:
            assistant_response, self.conversation_history = ask_gpt(
                model=self.model,
                system_prompt=self.character,
                question=message.content,
                conversation_history=self.conversation_history,
                max_history=3) 

            print(f"response: {assistant_response}")
            await message.channel.send(assistant_response)

client = ChatBot(intents=intents)
await client.start(DISCORD_TOKEN)
